In [177]:
# Data Preprocessing Package
import re
import numpy as np
import pandas as pd
import os
# NLP Package
from konlpy.tag import * 
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from collections import Counter

# Visualization Package
import pyLDAvis
#import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline


from pprint import pprint
import itertools
import math

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
 
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import nltk
#Data Preprocessing Package
import re
import numpy as np
import pandas as pd
import os
import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stop_words
import re
import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import WordPunctTokenizer 
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stop_words
import re
from nltk.tokenize import word_tokenize

In [178]:
ROOT_PATH ="C:/Users/user/Desktop/주혜/LAB/뉴스모델링"

# 데이터 저장 주소
DATA_FOLDER_PATH = os.path.join(ROOT_PATH,"Data/")
DATA_FILE_NAME = os.path.join(DATA_FOLDER_PATH,"ko_esg_2020to21.xlsx")    # Portal News Crawler에서 저장한 데이터 
DATA_STOP_WORD_FILE_NAME = os.path.join(DATA_FOLDER_PATH,"stopword.txt")   # 불용어 사전 


In [179]:
df= pd.read_excel(DATA_FILE_NAME)

In [180]:
stopword_vocab = DATA_STOP_WORD_FILE_NAME # 불용어 파일 불러오기

In [181]:
stop_words=[]

with open(stopword_vocab, encoding = 'utf-8') as f:
    for i in f:
        i = re.sub('\n','',i)
        stop_words.append(i)

In [182]:
dftext = df['text']
dftext

0        유튜브에 거짓말로 먹고사는사람들중\n대다수는 역사왜곡과 경제관련한 거짓말등이다. 미...
1        [ESG경영선언] SK, ‘친환경’ 총력…최태원式 비즈니스모델 혁신 기대감↑ htt...
2        ESG 테마 수혜 유망주 정리 - 골드만삭스 \n\n링크 👉 https://t.co...
3        정리: 테슬라가 대단한건 맞지만 일론을 찬양하는 빠순이가 되지는 말자. 테슬라한테만...
4        개인적으로는 테슬라주가의 퍼포먼스를 보며 일론머스크를 찬양하는 자들에게서는 배울게 ...
                               ...                        
13702    지금 막 접은 물개입니다. http://3.ly/ESG 에서 보고 접은 건데.. 동...
13703          폴 데이 “국가·기업별 ESG 표준화 시급” http://ff.im/7fHIz
13704          폴 데이 “국가·기업별 ESG 표준화 시급” http://ff.im/7e9d5
13705      투자자들, 이머징마켓 기업의 ESG정보공시 개선요구 http://ff.im/65R8n
13706    seri.org는 익스플로러기반으로 설계되어있어 FF사용시 자바스크립과 충돌일으켜 ...
Name: text, Length: 13707, dtype: object

In [183]:
stop_words

['[공유]',
 '정회원',
 '꼰대 ',
 '아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간'

In [184]:
len(stop_words)

679

In [185]:
pattern=re.compile(r'([\t\r()-.,:;!?`‘\'"�‧|%“])+')
tokenizer=TreebankWordTokenizer()
lemma = WordNetLemmatizer()

remove_bracket=['[',']','step']
stop_words.extend(remove_bracket)
len(stop_words)


682

In [190]:
def prepro(text):
    a=text.lower()
    b=re.sub(pattern,'',a)
    b=re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', '', b)
    c=re.sub('\n',' ',b)
    c=re.sub('@[^ \t\r\n\v\f]*',' ',c)
    c = re.sub('http[^ \t\r\n\v\f]*',' ',c)
    e_list=word_tokenize(c)
    result=[w for w in e_list if w not in stop_words]

    return result

In [191]:
cleaned = [prepro(i) for i in dftext]
cleanedtext = [' '.join(i) for i in cleaned]

In [192]:
cleanedtext

['유튜브에 거짓말로 먹고사는사람들중 대다수는 역사왜곡과 경제관련한 거짓말등이다 미국 독립과 관련한왜곡과 esg등 경제현안에대한 왜곡이다 거짓말하는 인물을 구글링하여보니 그렇게살지않으면 마땅한 생계수단이 없는 고학력실업자가대부분이었다서울대하버드유학해도실업자인분들이많다',
 'esg경영선언 sk 친환경 ’ 총력…최태원式 비즈니스모델 혁신 기대감↑',
 'esg 테마 수혜 유망주 정리 골드만삭스 링크 👉',
 '정리 테슬라가 대단한건 맞지만 일론을 찬양하는 빠순이가 되지는 말자 테슬라한테만 돈이 쏠릴거라고 생각하지 말자 esg 버블은 시작되었으니 안오른걸 찾는게 투자자로서는 돈을 더 많이 벌것이다 테슬라 자동차가 맨땅에서 굴러가겠냐',
 '개인적으로는 테슬라주가의 퍼포먼스를 보며 일론머스크를 찬양하는 자들에게서는 배울게 없다는 생각임 그린본드는결국 esg라는 딱지만 붙어도 돈이가는 닷컴버블과 같은 esg버블이 올것이라는 생각이 내 머리를 지배 난 시장에서 객관성을 잃은자들은 그냥 병신이라고 생각함',
 '문대통령 g20서 탄소중립 강조…공급망회의 참석 요즘 문의 진정한 뒷배경은 유럽 금융 자본 지나칠정도로 가속화 처음은 미국 금융자본이 받쳐주는 듯했지만 요즘 보면 유럽쪽이 압력이 더 세진듯한 아무리 esg 국제어젠다라고',
 '요새 꼴데케미칼이 갑자기 광고를 존나게 하는게 왜인가했더니 얘들 폐수 무단방류인가 해서 esg 점수 최하점 받았었구낰ㅋㅋㅋㅋㅋ',
 '공유 두나무 esg 경영 위해 1000억 투자한다 토큰포스트',
 '환경 패러다임 esg 경영에 올인하는 글로벌 기업들',
 '넘쳐나는 의류 폐기물 속 지구를 지킬 수 있는 염색 기법이 있다면 # 청소년기자 # 대학생기자 # 이하은기자 # 천연염색 # 친환경 # esg # 의류 # 환경 & gt 자세히보기',
 '넘쳐나는 의류 폐기물 속 지구를 지킬 수 있는 염색 기법이 있다면 # 청소년기자 # 대학생기자 # 이하은기자 # 천연염색 # 친환경 # esg # 의류 # 환경 & gt 자세히보기',
 '블록체인 기

In [195]:
df['cleaned_text']=cleanedtext

In [143]:
df.columns

Index(['Unnamed: 0', 'author_id', 'username', 'author_followers',
       'author_tweets', 'author_description', 'author_location', 'text',
       'created_at', 'retweets', 'replies', 'likes', 'quote_count', 'date',
       'cleaned_text'],
      dtype='object')

In [196]:
new_columns=['Unnamed: 0', 'author_id', 'username', 'author_followers',
       'author_tweets', 'author_description', 'author_location', 'text','cleaned_text',
       'created_at', 'retweets', 'replies', 'likes', 'quote_count', 'date']

In [197]:
df_n = df[new_columns]

In [175]:
df_n

,Unnamed: 0,author_id,username,author_followers,author_tweets,author_description,author_location,text,cleaned_text,created_at,retweets,replies,likes,quote_count,date
0,2,1415660936543621120,76JmVHXwchX2BAL,7,80,NaN,NaN,유튜브에 거짓말로 먹고사는사람들중\n대다수는 역사왜곡과 경제관련한 거짓말등이다. 미...,유튜브에 거짓말로 먹고사는사람들중 대다수는 역사왜곡과 경제관련한 거짓말등이다 미국 ...,2021-10-31 17:43:05,0,0,0,0,2021-10-31
1,7,1415215253483187968,BlogserT,0,13564,NaN,NaN,"[ESG경영선언] SK, ‘친환경’ 총력…최태원式 비즈니스모델 혁신 기대감↑ htt...",[ esg경영선언 ] sk 친환경’ 총력…최태원式 비즈니스모델 혁신 기대감↑,2021-10-31 15:55:04,0,0,0,0,2021-10-31
2,8,1287341462456160000,GONOGO_Korea,4885,8634,Investing Metaverse \n\nLet's #DOGE (Do Only G...,Decentralized World,ESG 테마 수혜 유망주 정리 - 골드만삭스 \n\n링크 👉 https://t.co...,esg 테마 수혜 유망주 정리 골드만삭스 링크 👉,2021-10-31 15:16:34,3,0,29,0,2021-10-31
3,9,1384144504148558080,min_tankmom,857,1434,매매보다 공부하는 시간이 많은 인간. 구 동시통역사 현 개인사업자. 관심사: 거시경...,NaN,정리: 테슬라가 대단한건 맞지만 일론을 찬양하는 빠순이가 되지는 말자. 테슬라한테만...,정리 테슬라가 대단한건 맞지만 일론을 찬양하는 빠순이가 되지는 말자 테슬라한테만 돈...,2021-10-31 14:30:58,6,2,50,0,2021-10-31
4,10,1384144504148558080,min_tankmom,857,1434,매매보다 공부하는 시간이 많은 인간. 구 동시통역사 현 개인사업자. 관심사: 거시경...,NaN,개인적으로는 테슬라주가의 퍼포먼스를 보며 일론머스크를 찬양하는 자들에게서는 배울게 ...,개인적으로는 테슬라주가의 퍼포먼스를 보며 일론머스크를 찬양하는 자들에게서는 배울게 ...,2021-10-31 14:08:08,25,14,95,15,2021-10-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,17515,46826529,ranghes,1049,28334,크리스마스 직전에 성모자애보육원에 장난감 보내려고 하고 있습니다. 동참을 희망하시면...,"Seoul, Korea",지금 막 접은 물개입니다. http://3.ly/ESG 에서 보고 접은 건데.. 동...,지금 막 접은 물개입니다 http//3ly/esg 보고 접은 건데 동영상으로 되어 ...,2009-08-29 04:38:06,0,0,0,0,2009-08-29
17329,17516,15929033,wade_paak,392,7790,New project out now 👉 https://t.co/VQfG74r2R1,"Los Angeles, CA",폴 데이 “국가·기업별 ESG 표준화 시급” http://ff.im/7fHIz,폴 데이 국가·기업별 표준화 시급” http//ffim/7fhiz,2009-08-27 19:08:03,0,0,0,0,2009-08-27
17330,17517,15929033,wade_paak,392,7790,New project out now 👉 https://t.co/VQfG74r2R1,"Los Angeles, CA",폴 데이 “국가·기업별 ESG 표준화 시급” http://ff.im/7e9d5,폴 데이 국가·기업별 표준화 시급” http//ffim/7e9d5,2009-08-27 09:28:14,0,0,0,0,2009-08-27
17331,17518,52312062,sustinvest,179,4955,사회책임투자 리서치&컨설팅 _x000D_\n서스틴베스트입니다.,"Seoul, Korea","투자자들, 이머징마켓 기업의 ESG정보공시 개선요구 http://ff.im/65R8n",투자자들 이머징마켓 기업의 http//ffim/65r8n,2009-08-03 08:55:44,0,0,0,0,2009-08-03


In [198]:
df_n.to_excel('korean_twit_cleaned.xlsx')